In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.base import BaseEstimator, RegressorMixin

In [2]:
# Loads data.
boston = load_boston()
print(boston.data.shape)

(506, 13)


In [3]:
# Standardizes data.
mean = np.mean(boston.data, axis=0)
std = np.std(boston.data, axis=0)
A = (boston.data - mean)/std
b = boston.target

In [4]:
class Admm(BaseEstimator, RegressorMixin):
    def __init__(self, lambd=1.0, rho=1.0, max_iter=1000):
        self.lambd = lambd
        self.rho = rho
        self.threshold = lambd / rho
        self.max_iter = max_iter
        self.coef_ = None
        self.intercept_ = 0.0

    def _soft_threshold(self, x):
        t = self.threshold
        
        positive_indexes = x >= t
        negative_indexes = x <= t
        zero_indexes = abs(x) <= t
        
        y = np.zeros(x.shape)    
        y[positive_indexes] = x[positive_indexes] - t
        y[negative_indexes] = x[negative_indexes] + t
        y[zero_indexes] = 0.0
    
        return y

    def fit(self, A, b):
        N = A.shape[0]
        M = A.shape[1]
        inv_matrix = np.linalg.inv(np.dot(A.T, A) / N + self.rho * np.identity(M))
        
        x = np.dot(A.T, b) / N
        z = x.copy()
        y = np.zeros(len(x))
    
        for iteration in range(self.max_iter):
            x = np.dot(inv_matrix, np.dot(A.T, b) / N + self.rho * z - y)
            z = self._soft_threshold(x + y / self.rho)
            y += self.rho * (x - z)

        self.coef_ = x
                      
        return self

    def predict(self, X):
        y = np.dot(X, self.coef_)
        return y

In [5]:
np.set_printoptions(formatter={'float': '{: 0.10f}'.format})

In [6]:
model = Admm(lambd=1.0, rho=1.0, max_iter=1000)
model.fit(A, b)
print("Lasso by ADMM")
print(model.coef_)

Lasso by ADMM
[-0.0000000000  0.0000000000 -0.0000000000 -0.0000000000  0.0000000000
  2.7131072809  0.0000000000  0.0000000000  0.0000000000  0.0000000000
 -1.3434986189  0.1807938799 -3.5436116588]


In [7]:
from sklearn import linear_model

model = linear_model.Lasso(alpha=1.0, max_iter=1000)
model.fit(A, b)
print("Lasso by sickit-learn (coordinate descent)")
print(model.coef_)

Lasso by sickit-learn (coordinate descent)
[-0.0000000000  0.0000000000 -0.0000000000  0.0000000000 -0.0000000000
  2.7133533427 -0.0000000000 -0.0000000000 -0.0000000000 -0.0000000000
 -1.3435484031  0.1809553656 -3.5433828848]
